# Enter State Farm

If you want some tutorial, follow the video from Lesson 48:50

### Import necessary libraries

In [2]:
from theano.sandbox import cuda
cuda.use('gpu1')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [3]:
%matplotlib inline
from __future__ import division, print_function
from importlib import reload
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

path = 'data/state/sample/'
batch_size = 64

In [5]:
import os

First, let's get the data ready.

Move to the data folder and create a new directory for statefarm:

```bash
cd ~/fastai-notes/deeplearning1/nbs/data/
mkdir state
cd state
```

Download the data from kaggle. Note: don't forget to accept the competition rule.
```bash
kg download -u yingchi.pei@gmail.com -p FL199473/kag -c state-farm-distracted-driver-detection
```
Unzip the downloaded data:
```bash
unzip -q imgs.zip
unzip -q driver_imgs_list.csv.zip
```

### Get the data ready

#### Create the validation set

Count the number of files in the current directory: `find . -type f | wc -l`

In each category's folder inside /train, there are around 2.2K images. So let's take 200 from each category to the validation set.

In [13]:
DATA_HOME_DIR='/home/ubuntu/fastai-notes/deeplearning1/nbs/data/state'
%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%cd train

/home/ubuntu/fastai-notes/deeplearning1/nbs/data/state
mkdir: cannot create directory ‘valid’: File exists
mkdir: cannot create directory ‘results’: File exists
/home/ubuntu/fastai-notes/deeplearning1/nbs/data/state/train


In [11]:
for d in glob('c?'): os.mkdir('../valid/'+d)

In [14]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i], DATA_HOME_DIR+'/valid/'+shuf[i])

#### Create the sample set

In [16]:
%cd $DATA_HOME_DIR

/home/ubuntu/fastai-notes/deeplearning1/nbs/data/state


In [18]:
%mkdir -p sample/train
%mkdir -p sample/valid

In [19]:
%cd train

/home/ubuntu/fastai-notes/deeplearning1/nbs/data/state/train


In [20]:
from shutil import copyfile

for d in glob('c?'): 
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [21]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/'+shuf[i])

In [22]:
%cd ../valid

/home/ubuntu/fastai-notes/deeplearning1/nbs/data/state/valid


In [23]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/'+shuf[i])

In [24]:
%cd ../../..
%mkdir data/state/sample/test

/home/ubuntu/fastai-notes/deeplearning1/nbs


### Create batches

In [25]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [26]:
??get_classes()

In [27]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


### Basic models

#### Linear model

First, we try the simplest model and use default parameter. 

Note the trick of making the first layer a batchnorm layer - so that we don't have to worry about normalizing the input.

In [28]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3, 224, 224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

In [30]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2,
                   validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 13.6292 - acc: 0.1500 - val_loss: 13.3279 - val_acc: 0.1700
Epoch 2/2
1500/1500 [==============================] - 24s - loss: 14.3084 - acc: 0.1107 - val_loss: 13.7446 - val_acc: 0.1460


Ya, this model training is going nowhere...
Let's check the number of parameters in our model

In [31]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 224, 224)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 150528)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            1505290     flatten_1[0][0]                  
Total params: 1,505,302
Trainable params: 1,505,296
Non-trainable params: 6
____________________________________________________________________________________________________


Over 1.5 million parameters - that should be enough. Incidentally, it's worth checking you understand why this is the number of parameters in this layer:

In [32]:
10*3*224*224

1505280

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts 1 or 2 classes wigh high confidence, so that it can give a 0 prediction to as many classes as possible.

In [34]:
np.round(model.predict_generator(batches, batches.N)[:20], 2)

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

Our hypothesis was correct. It's nearly always predicting class 1 or 9. 

From keras documentation, the default params for Adam() is:
`Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)`

So let's try a lower learning rate:

In [38]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3, 224, 224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 32s - loss: 4.6020 - acc: 0.2793 - val_loss: 8.0088 - val_acc: 0.2700
Epoch 2/4
1500/1500 [==============================] - 24s - loss: 2.3559 - acc: 0.6407 - val_loss: 3.1900 - val_acc: 0.5210
Epoch 3/4
1500/1500 [==============================] - 24s - loss: 1.8422 - acc: 0.7927 - val_loss: 2.2388 - val_acc: 0.7370
Epoch 4/4
1500/1500 [==============================] - 24s - loss: 1.6826 - acc: 0.8467 - val_loss: 2.4954 - val_acc: 0.6940


We're stabilizing at validation accuracy of 0.7. Much better than a random guess!

Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:

In [39]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 1000 images belonging to 10 classes.


In [43]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]
np.round(val_res, 2)

array([[ 2.58,  0.68],
       [ 2.38,  0.69],
       [ 2.71,  0.67],
       [ 2.46,  0.71],
       [ 2.53,  0.68],
       [ 2.56,  0.68],
       [ 2.49,  0.69],
       [ 2.64,  0.67],
       [ 2.44,  0.71],
       [ 2.63,  0.67]])